In [3]:
from pyspark.sql import SparkSession

In [4]:
spark=SparkSession\
.builder.\
appName("python spark sql example")\
.config("spark.some.config.option","some-value")\
.getOrCreate()

# LR

## Binomial logistic regression

In [1]:
from  pyspark.ml.classification import LogisticRegression

In [5]:
training=spark.read.format('libsvm').load('data/mllib/sample_libsvm_data.txt')

In [38]:
type(training)

pyspark.sql.dataframe.DataFrame

In [158]:
# training.select('label').show()

In [20]:
lr=LogisticRegression(maxIter=10,regParam=0.3,elasticNetParam=0.8)

In [21]:
lrModel=lr.fit(training)

In [36]:
training['features']

<bound method _bin_op.<locals>._ of Column<b'features'>>

In [44]:
pre=lrModel.transform(training.select('features'))

In [159]:
pre.toPandas().head(10)

,features,rawPrediction,probability,prediction
0,"(-0.222222, 0.5, -0.762712, -0.833333)","[-0.21305451012206839, 0.7854380421234279, 0.0...","[0.19824091021950388, 0.5380629504663061, 0.26...",1.0
1,"(-0.555556, 0.25, -0.864407, -0.916667)","[-0.23952541514793146, 0.8962139249724502, 0.0...","[0.18250386256254247, 0.568221142106847, 0.249...",1.0
2,"(-0.722222, -0.166667, -0.864407, -0.833333)","[-0.21305451012206839, 0.8648002451366628, 0.0...","[0.18980556250236028, 0.5577188309398853, 0.25...",1.0
3,"(-0.722222, 0.166667, -0.694915, -0.916667)","[-0.23952541514793146, 0.7639433264856103, 0.0...","[0.19632523546632502, 0.5355216883905428, 0.26...",1.0
4,"(0.166667, -0.416667, 0.457627, 0.5)","[0.21047647616023052, -0.6695223444410741, 0.0...","[0.4375039818343829, 0.18146938948218602, 0.38...",0.0
5,"(-0.833333, 0.0, -0.864407, -0.916667)","[-0.23952541514793146, 0.8962139249724502, 0.0...","[0.18250386256254247, 0.568221142106847, 0.249...",1.0
6,"(-1.32455e-07, -0.166667, 0.220339, 0.0833333)","[0.07812299927036823, -0.32727697566780806, 0....","[0.37581775428218006, 0.2505614888540287, 0.37...",0.0
7,"(-1.32455e-07, -0.333333, 0.0169491, -4.03573e...","[0.05165230377890003, -0.1371392165046517, 0.0...","[0.3510273915379525, 0.2906363236580725, 0.358...",2.0
8,"(-0.5, 0.75, -0.830508, -1.0)","[-0.2659960025254754, 0.9011726399131196, 0.07...","[0.17808226409449213, 0.5721574660564578, 0.24...",1.0
9,"(0.611111, 0.0, 0.694915, 0.416667)","[0.18400588878268653, -0.8232872551325279, 0.0...","[0.44258017540583633, 0.16163302558940845, 0.3...",0.0


## Multinomial logistic regression

In [160]:
from pyspark.ml.classification import LogisticRegression

# Load training data
training = spark \
    .read \
    .format("libsvm") \
    .load("data/mllib/sample_multiclass_classification_data.txt")

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(training)

# Print the coefficients and intercept for multinomial logistic regression
print("Coefficients: \n" + str(lrModel.coefficientMatrix))
print("Intercept: " + str(lrModel.interceptVector))

trainingSummary = lrModel.summary

# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)

# for multiclass, we can inspect metrics on a per-label basis
print("False positive rate by label:")
for i, rate in enumerate(trainingSummary.falsePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

print("True positive rate by label:")
for i, rate in enumerate(trainingSummary.truePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

print("Precision by label:")
for i, prec in enumerate(trainingSummary.precisionByLabel):
    print("label %d: %s" % (i, prec))

print("Recall by label:")
for i, rec in enumerate(trainingSummary.recallByLabel):
    print("label %d: %s" % (i, rec))

print("F-measure by label:")
for i, f in enumerate(trainingSummary.fMeasureByLabel()):
    print("label %d: %s" % (i, f))

accuracy = trainingSummary.accuracy
falsePositiveRate = trainingSummary.weightedFalsePositiveRate
truePositiveRate = trainingSummary.weightedTruePositiveRate
fMeasure = trainingSummary.weightedFMeasure()
precision = trainingSummary.weightedPrecision
recall = trainingSummary.weightedRecall
print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

Coefficients: 
3 X 4 CSRMatrix
(0,3) 0.3176
(1,2) -0.7804
(1,3) -0.377
Intercept: [0.05165231659832854,-0.12391224990853622,0.07225993331020768]
objectiveHistory:
1.098612288668108
1.087602085441699
1.0341156572156232
1.0289859520256006
1.0300389657358995
1.0239965158223991
1.0236097451839508
1.0231082121970012
1.023022220302788
1.0230018151780262
1.0229963739557606
False positive rate by label:
label 0: 0.22
label 1: 0.05
label 2: 0.0
True positive rate by label:
label 0: 1.0
label 1: 1.0
label 2: 0.46
Precision by label:
label 0: 0.6944444444444444
label 1: 0.9090909090909091
label 2: 1.0
Recall by label:
label 0: 1.0
label 1: 1.0
label 2: 0.46
F-measure by label:
label 0: 0.819672131147541
label 1: 0.9523809523809523
label 2: 0.6301369863013699
Accuracy: 0.82
FPR: 0.09
TPR: 0.82
F-measure: 0.8007300232766211
Precision: 0.8678451178451179
Recall: 0.82


# Decision tree classifier

In [161]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load the data stored in LIBSVM format as a DataFrame.
data = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))

treeModel = model.stages[2]
# summary only
print(treeModel)

+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       1.0|         1.0|(692,[95,96,97,12...|
|       1.0|         1.0|(692,[98,99,100,1...|
|       1.0|         1.0|(692,[100,101,102...|
|       1.0|         1.0|(692,[121,122,123...|
|       1.0|         1.0|(692,[122,123,124...|
+----------+------------+--------------------+
only showing top 5 rows

Test Error = 0 
DecisionTreeClassificationModel (uid=DecisionTreeClassifier_440d97e19b4f4a2c3108) of depth 2 with 5 nodes


# Random forest classifier

In [162]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load and parse the data file, converting it to a DataFrame.
data = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("predictedLabel", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

rfModel = model.stages[2]
print(rfModel)  # summary only

+--------------+-----+--------------------+
|predictedLabel|label|            features|
+--------------+-----+--------------------+
|           0.0|  0.0|(692,[95,96,97,12...|
|           0.0|  0.0|(692,[98,99,100,1...|
|           0.0|  0.0|(692,[100,101,102...|
|           0.0|  0.0|(692,[124,125,126...|
|           0.0|  0.0|(692,[126,127,128...|
+--------------+-----+--------------------+
only showing top 5 rows

Test Error = 0
RandomForestClassificationModel (uid=RandomForestClassifier_49669744d93035453aa5) with 10 trees


# Gradient-boosted tree classifier

In [163]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load and parse the data file, converting it to a DataFrame.
data = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a GBT model.
gbt = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", maxIter=10)

# Chain indexers and GBT in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, gbt])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

gbtModel = model.stages[2]
print(gbtModel)  # summary only

+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       1.0|         1.0|(692,[95,96,97,12...|
|       1.0|         1.0|(692,[98,99,100,1...|
|       1.0|         1.0|(692,[123,124,125...|
|       1.0|         1.0|(692,[124,125,126...|
|       1.0|         1.0|(692,[124,125,126...|
+----------+------------+--------------------+
only showing top 5 rows

Test Error = 0
GBTClassificationModel (uid=GBTClassifier_4f1f9905e1bf8a2f9e25) with 10 trees


# Multilayer perceptron classifier

In [164]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [165]:
data = spark.read.format("libsvm")\
    .load("data/mllib/sample_multiclass_classification_data.txt")

# Split the data into train and test
splits = data.randomSplit([0.6, 0.4], 1234)
train = splits[0]
test = splits[1]


In [169]:
layers=[4,4,3,3]

In [170]:
trainer=MultilayerPerceptronClassifier(maxIter=100,\
        layers=layers,
        blockSize=128,
        seed=1234)

In [174]:
train.toPandas().head(10)

,label,features
0,0.0,"(-0.166667, -0.416667, 0.38983, 0.5)"
1,0.0,"(-0.166667, -0.416667, 0.38983, 0.5)"
2,0.0,"(-0.166667, -0.333333, 0.38983, 0.916667)"
3,0.0,"(-0.111111, -0.166667, 0.38983, 0.416667)"
4,0.0,"(-1.32455e-07, -0.5, 0.559322, 0.0833333)"
5,0.0,"(0.0555554, -0.333333, 0.288136, 0.416667)"
6,0.0,"(0.0555554, 0.166667, 0.491525, 0.833333)"
7,0.0,"(0.111111, -0.416667, 0.322034, 0.416667)"
8,0.0,"(0.111111, -0.333333, 0.38983, 0.166667)"
9,0.0,"(0.111111, -0.25, 0.559322, 0.416667)"


In [171]:
model=trainer.fit(train)

In [175]:
result=model.transform(test)

In [176]:
predictionAndLabels=result.select("prediction","label")

In [177]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

In [178]:
evaluator.evaluate(predictionAndLabels)

0.8823529411764706

# Linear Support Vector Machine

In [184]:
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [180]:
training=spark.read.format("libsvm")\
        .load("data/mllib/sample_libsvm_data.txt")

In [181]:
lsvc=LinearSVC(maxIter=10,regParam=0.1)

In [182]:
lsvcModel=lsvc.fit(training)

In [183]:
pre=lsvcModel.transform(training)

In [186]:
predictionAndLabels=result.select("prediction","label")

In [187]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

In [188]:
evaluator.evaluate(predictionAndLabels)

0.8823529411764706

# One-vs-Rest classifier

In [189]:
from pyspark.ml.classification import LogisticRegression,OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [190]:
inputData = spark.read.format("libsvm") \
    .load("data/mllib/sample_multiclass_classification_data.txt")

In [191]:
(train,test)=inputData.randomSplit([0.8,0.2])

In [192]:
lr=LogisticRegression(maxIter=10,tol=1E-6,fitIntercept=True)

In [193]:
ovr=OneVsRest(classifier=lr)

In [194]:
ovrModel=ovr.fit(train)

In [195]:
ovrModel.classifier

Param(parent='OneVsRestModel_4b1fbcd7483921c4f954', name='classifier', doc='base binary classifier')

In [196]:
predictions = ovrModel.transform(test)

In [197]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

In [198]:
accuracy = evaluator.evaluate(predictions)

In [199]:
accuracy

1.0

# Naive Bayes

In [200]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [201]:
data = spark.read.format("libsvm") \
    .load("data/mllib/sample_libsvm_data.txt")

In [202]:
splits=data.randomSplit([0.6,0.4],1234)

In [203]:
train=splits[0]
test=splits[1]

In [204]:
nb=NaiveBayes(smoothing=1.0,modelType="multinomial")

In [205]:
model=nb.fit(train)

In [206]:
predictions=model.transform(test)

In [207]:
predictions.show()

+-----+--------------------+--------------------+-----------+----------+
|label|            features|       rawPrediction|probability|prediction|
+-----+--------------------+--------------------+-----------+----------+
|  0.0|(692,[95,96,97,12...|[-174115.98587057...|  [1.0,0.0]|       0.0|
|  0.0|(692,[98,99,100,1...|[-178402.52307196...|  [1.0,0.0]|       0.0|
|  0.0|(692,[100,101,102...|[-100905.88974016...|  [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|[-244784.29791241...|  [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|[-196900.88506109...|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[-238164.45338794...|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[-184206.87833381...|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|[-214174.52863813...|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|[-182844.62193963...|  [1.0,0.0]|       0.0|
|  0.0|(692,[128,129,130...|[-246557.10990301...|  [1.0,0.0]|       0.0|
|  0.0|(692,[152,153,154...|[-208282.08496711...|  

In [208]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", 
    predictionCol="prediction",                     
    metricName="accuracy")

In [209]:
accuracy=evaluator.evaluate(predictions)

In [210]:
accuracy

1.0